Detección de caras con webcam

In [3]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [24]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 1
imodoE = 0

debug = 0

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break


# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0


In [1]:
import cv2
import time
import threading
from FaceDetectors import FaceDetector
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
# Función para la captura de video en un hilo
def video_capture_thread():
    global cap, frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break

# Función para el procesamiento de video en un hilo
def video_processing_thread():
    global frame
    while True:
        t = time.time()
        if frame is not None:
            # Realizar el procesamiento en este hilo
            B, G, R = cv2.split(frame)
                # Search face with a specific setup for face and eye detection
            values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
            print(values)
            if values is not None:
                face, eyes, shape = values

                #draws face container
                [x, y , w, h] = face
                if x > -1:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                    # draws eyes and mask if available
                    [lex, ley, rex, rey] = eyes
                    if lex > -1:
                        # Show detected facial elements
                        if imodoF > 0:
                            for (x, y) in shape:
                                cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                        cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                        cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                        # Normalize and show
                        # color channels
                        normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                        Bnorm = normalizatorHS.normf_image
                        normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                        Gnorm = normalizatorHS.normf_image
                        normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                        Rnorm = normalizatorHS.normf_image
                        NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                        cv2.imshow("Normalized", NormBGR)


            if debug:
                print("Processing time : {:.3f}".format(time.time() - t))

# Inicializar las variables globales
frame = None
debug = 0

normalizatorHS = faceutils.Normalization()
FDet = FaceDetector()
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Iniciar los hilos para la captura y el procesamiento de video
capture_thread = threading.Thread(target=video_capture_thread)
processing_thread = threading.Thread(target=video_processing_thread)

capture_thread.start()
processing_thread.start()

# Esperar a que ambos hilos terminen
capture_thread.join()
processing_thread.join()

# Liberar la cámara y cerrar ventanas
cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-6 (video_processing_thread):
Traceback (most recent call last):
  File "c:\Users\Eric\anaconda3\envs\VC_P1\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\Eric\anaconda3\envs\VC_P1\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Eric\AppData\Local\Temp\ipykernel_4428\3321155088.py", line 25, in video_processing_thread
NameError: name 'imodoF' is not defined


In [ ]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors


# Normalizador HS
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Ruta de la imagen
image_path = 'ryan-gosling-1655362981.jpg'  # Reemplaza con la ruta de tu imagen


# Carga la imagen
image = cv2.imread(image_path)
# Carga la imagen
frame = cv2.imread(image_path)

# Obtiene las dimensiones de la imagen
height, width, _ = image.shape

# Crea una ventana con el tamaño de la imagen
cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Image', 2200, 1400)


B, G, R = cv2.split(frame)
# Detección de rostros y ojos
values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
if values is not None:
    print(values)
    face, eyes, shape = values

    # Dibuja el contenedor del rostro
    [x, y, w, h] = face
    if x > -1:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Dibuja los ojos y la máscara si están disponibles
        [lex, ley, rex, rey] = eyes
        if lex > -1:
            # Muestra los elementos faciales detectados
            for (x, y) in shape:
                cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

            cv2.circle(frame, (int(lex), int(ley)), 4, (0, 0, 255), -1)
            cv2.circle(frame, (int(rex), int(rey)), 4, (0, 255, 0), -1)

            # Normaliza y muestra
            # canales de color
            normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
            Bnorm = normalizatorHS.normf_image
            normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
            Gnorm = normalizatorHS.normf_image
            normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
            Rnorm = normalizatorHS.normf_image
            NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))

# Muestra la imagen resultante
cv2.putText(frame, FDet.FaceDetectors[1], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.imshow('Prueba Gosling', frame)
cv2.waitKey(0)  # Espera hasta que se presione cualquier tecla
cv2.destroyAllWindows()


([683, 286, 445, 446], [828.6666666666666, 377.0, 1011.8333333333334, 391.0], array([[ 667,  356],
       [ 667,  429],
       [ 673,  504],
       [ 682,  576],
       [ 705,  644],
       [ 747,  704],
       [ 795,  756],
       [ 851,  798],
       [ 913,  810],
       [ 971,  798],
       [1019,  756],
       [1060,  707],
       [1094,  652],
       [1112,  591],
       [1122,  526],
       [1131,  460],
       [1136,  394],
       [ 746,  346],
       [ 778,  322],
       [ 817,  312],
       [ 859,  316],
       [ 896,  331],
       [ 973,  331],
       [1010,  316],
       [1048,  313],
       [1084,  328],
       [1101,  361],
       [ 931,  386],
       [ 932,  431],
       [ 934,  477],
       [ 935,  525],
       [ 876,  541],
       [ 900,  554],
       [ 926,  567],
       [ 950,  559],
       [ 970,  548],
       [ 791,  378],
       [ 816,  363],
       [ 843,  365],
       [ 867,  385],
       [ 841,  386],
       [ 814,  385],
       [ 974,  396],
       [ 999,  378]

In [8]:
import cv2
import numpy as np
import FaceNormalizationUtils as faceutils
import FaceDetectors

# Normalizador HS
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Ruta de la imagen
image_path = 'ryan-gosling-1655362981.jpg'  # Reemplaza con la ruta de tu imagen

# Carga la imagen
image = cv2.imread(image_path)

# Detección de la cara en la imagen
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])

if faces is not None and len(faces) > 0:
    x, y, w, h = faces[0]  # Suponemos que solo hay una cara en la imagen
    face_image = image[y:y+h, x:x+w]

# Inicializa la cámara
cap = cv2.VideoCapture(0)  # 0 para la cámara por defecto, puedes ajustarlo si tienes múltiples cámaras

while True:
    # Captura un fotograma de la cámara
    ret, frame = cap.read()
    B, G, R = cv2.split(frame)
    if not ret:
        break

    # Detección de rostros y ojos en el video de la cámara
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
    if values is not None:
        face, eyes, shape = values

        # Dibuja el contenedor del rostro
        [x, y, w, h] = face
        if x > -1:
            # Dibuja el contenedor de la cara detectada
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Muestra los elementos faciales detectados
            for (x, y) in shape:
                cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

            # Normaliza y muestra canales de color
            lex, ley, rex, rey = eyes
            if lex > -1:
                # Normaliza la cara detectada en tiempo real
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)

    # Superpone el marco de la cara de la imagen en el video de la cámara
    if face_image is not None:
       face_image = frame[y:y+h, x:x+w] 

    # Muestra la imagen resultante
    cv2.imshow('Webcam Face Detection', frame)

    # Sale del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera la cámara y destruye todas las ventanas de OpenCV
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [ ]:
import cv2
import numpy as np
import FaceNormalizationUtils as faceutils
import FaceDetectors
import matplotlib.pyplot as plt
# Normalizador HS
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Ruta de la imagen
image_path = 'ryan-gosling-1655362981.jpg'  # Reemplaza con la ruta de tu imagen

# Carga la imagen
image = cv2.imread(image_path)

# Detección de la cara en la imagen
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])

if faces is not None and len(faces) > 0:
    x, y, w, h = faces[0]  # Suponemos que solo hay una cara en la imagen
    face_image = image[y:y+h, x:x+w]

cv2.imshow('Webcam Face Detection', face_image)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


Prueba superposicion

In [4]:
import cv2
import numpy as np
import FaceNormalizationUtils as faceutils
import FaceDetectors
import matplotlib.pyplot as plt

# Normalizador HS
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Ruta de la imagen de la cara que deseas superponer
image_path = 'ryan-gosling-1655362981.jpg'
# Carga la imagen de la cara
face_image = cv2.imread(image_path)

# Inicializa la captura de la webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Detección de rostros en el frame de la webcam
    
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])    
    
    if values is not None:
        face = values[0]
        x, y, w, h = face

        # Ajusta el tamaño de la imagen de la cara para que coincida con el tamaño de la cara detectada
        resized_face_image = cv2.resize(face_image, (w, h))
        
        # Superpone la cara en el frame de la webcam
        frame[y:y+h, x:x+w] = resized_face_image

    cv2.imshow('Webcam', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Presiona Esc para salir
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4065: error: (-215:Assertion failed) inv_scale_x > 0 in function 'cv::resize'


Prueba 2

In [31]:
import cv2
import numpy as np
import FaceNormalizationUtils as faceutils
import FaceDetectors
import matplotlib.pyplot as plt
import random
import time

# Normalizador HS
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Ruta de la imagen de la cara que deseas superponer
image_path = 'ryan-gosling-1655362981.jpg'
# Carga la imagen de la cara
face_image = cv2.imread(image_path)

if face_image is None:
    print(f"Error: Unable to load face image from {image_path}")
    exit(1)

# Inicializa la captura de la webcam
cap = cv2.VideoCapture(0)

# Variables para el manejo de la ruleta de imágenes
image_paths = ['lalaland.jpg.jpg', 'blade_runner.jpg.jpg', 'ryan-gosling-1655362981.jpg']  # Agrega las rutas de tus imágenes
current_image_index = 0
last_image_change_time = time.time()
image_change_interval = 5  # Cambia la imagen cada 5 segundos

# Initialize roulette_image before entering the while loop
roulette_image = cv2.imread(image_paths[current_image_index])

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Detección de rostros en el frame de la webcam
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])

    if values is not None:
        face = values[0]
        x, y, w, h = face
        if w > 0 and h > 0:
            # Ajusta el tamaño de la imagen de la cara para que coincida con el tamaño de la cara detectada
            resized_face_image = cv2.resize(face_image, (w, h))
            # Superpone la cara en el frame de la webcam
            frame[y:y+h, x:x+w] = resized_face_image

    # Mostrar la imagen de la ruleta y cambiar la imagen después de cierto intervalo de tiempo
    if time.time() - last_image_change_time >= image_change_interval:
        current_image_index = random.randint(0, len(image_paths) - 1)
        current_image_path = image_paths[current_image_index]
        roulette_image = cv2.imread(current_image_path)
        last_image_change_time = time.time()

    # Ensure roulette_image is not None before accessing its shape
    if roulette_image is not None:
        frame[0:roulette_image.shape[0], 0:roulette_image.shape[1]] = roulette_image

    cv2.imshow('Webcam', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Presiona Esc para salir
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

prueba imagen arriba cara

In [45]:
import cv2
import numpy as np
import FaceNormalizationUtils as faceutils
import FaceDetectors
import random
import time

# Global variables
changing_images = False
last_image_change_time = time.time()
image_change_duration = 10  # Change the image every 10 seconds

# Function to handle mouse click events
def on_mouse_click(event, x, y, flags, param):
    global changing_images, last_image_change_time
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button click
        changing_images = True
        last_image_change_time = time.time()

# Normalizador HS
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Inicializa la captura de la webcam
cap = cv2.VideoCapture(0)

# Variables para el manejo de la ruleta de imágenes
image_paths = ['lalaland.jpg', 'blade_runner.jpg', 'ryan-gosling-1655362981.jpg']  # Agrega las rutas de tus imágenes
current_image_index = 0

# Initialize roulette_image before entering the while loop
roulette_image = cv2.imread(image_paths[current_image_index])
last_image_change_time = time.time()
# Create a window to capture mouse events
cv2.namedWindow('Webcam')
cv2.setMouseCallback('Webcam', on_mouse_click)

while True:
    current_time = time.time()
    ret, frame = cap.read()

    if not ret:
        break

    # Detección de rostros en el frame de la webcam
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])

    if values is not None:
        face = values[0]
        x, y, w, h = face
        if w > 0 and h > 0:
            # Resize the face image to match the size of the detected face
            resized_face_image = cv2.resize(face_image, (w, h))

            # Calculate the position for the superimposed face image above the detected face
            y_offset = max(0, y - h)  # Place it above the detected face
            x_offset = x

            # Superpose the resized face image on the frame
            frame[y_offset:y_offset + h, x_offset:x_offset + w] = resized_face_image

    if changing_images:
        if time.time() - last_image_change_time >= image_change_duration:
            # Change the image to a random one from image_paths
            current_image_index = random.randint(0, len(image_paths) - 1)
            current_image_path = image_paths[current_image_index]
            roulette_image = cv2.imread(current_image_path)

            changing_images = False  # Reset the flag
            last_image_change_time = time.time()  # Update the time of the last image change

    # Ensure roulette_image is not None before accessing its shape
    if roulette_image is not None:
        frame[0:roulette_image.shape[0], 0:roulette_image.shape[1]] = roulette_image

    cv2.imshow('Webcam', frame)

    key = cv2.waitKey(1)
    if key & 0xFF == 27:  # Press Esc to exit
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
import FaceNormalizationUtils as faceutils
import FaceDetectors
import random
import time
import pygame

# Global variables
changing_images = False
last_image_change_time = time.time()
image_change_duration = 10  # Change the image every 10 seconds
current_image_index = 0
time_remaining = image_change_duration  # Initialize time_remaining

# Function to handle mouse click events
def on_mouse_click(event, x, y, flags, param):
    global changing_images, last_image_change_time, time_remaining
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button click
        changing_images = True
        last_image_change_time = time.time()
        time_remaining = image_change_duration  # Reset time_remaining

# Normalizador HS
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Inicializa la captura de la webcam
cap = cv2.VideoCapture(0)

# Variables para el manejo de la ruleta de imágenes
image_paths = ['lalaland.jpg', 'blade.png', 'drive.png','ryan-gosling-1655362981.jpg']  
songs_paths =[
    "Y2meta.app - 'City of Stars' (Duet ft. Ryan Gosling, Emma Stone) - La La Land Original Motion Picture Soundtrack (128 kbps).mp3",
    'you-look-lonely-i-can-fix-that-128-ytshorts.savetube.me.mp3',
    'Y2meta.app - Kavinsky - Nightcall letra Ingles_Español (128 kbps).mp3',
    "Y2meta.app - Ryan Gosling - I'm Just Ken (From Barbie The Album) [Official Audio] (128 kbps).mp3",

]

# Initialize roulette_image before entering the while loop
roulette_image = cv2.imread(image_paths[current_image_index])

pygame.mixer.init()

# Create a window to capture mouse events
cv2.namedWindow('Webcam')
cv2.setMouseCallback('Webcam', on_mouse_click)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Calculate time remaining until image change
    time_remaining = max(0, image_change_duration - (time.time() - last_image_change_time))

    # Detección de rostros en el frame de la webcam
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
  
    if values is not None:
        face = values[0]
        x, y, w, h = face
        if w > 0 and h > 0:
            if roulette_image is not None:
                print("dentro")
                print(roulette_image)
   
                # Calculate the position for the superimposed face image above the detected face
                y_offset = max(0, y - h)  # Place it above the detected face
                x_offset = x

                             # Resize the face image to match the size of the detected face
                resized_face_image = cv2.resize(roulette_image, (w, h))

                cv2.putText(frame, f"Which Ryan Gosling are you?", (x_offset-125, y_offset-25), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)
                # Superpose the resized face image on the frame
                frame[y_offset:y_offset + h, x_offset:x_offset + w] = resized_face_image

    
    if changing_images:

        if current_image_index == len(image_paths):
            current_image_index = 0
        if time_remaining == 0:
            print("THE TIME HAS COME")
            print("--------------------------------------")
            print("Indice")
            print(current_image_index)
            
            print("--------------------------------------")
            changing_images = False
            last_image_change_time = time.time()
            current_image_index = random.randint(0, len(image_paths)-1)
            pygame.mixer.music.load(songs_paths[current_image_index])  # Replace with your music file
            pygame.mixer.music.play(-1)
        print("AQUI LLEGAMOS JEFE")
        print("--------------------------------------")
        print("Indice")
        print(current_image_index)
        
        print(image_paths[current_image_index])
        roulette_image = cv2.imread(image_paths[current_image_index])
        
        print(roulette_image)

        
        
        print(roulette_image)
        current_image_index += 1
        print("--------------------------------------")

            

    cv2.imshow('Webcam', frame)

    key = cv2.waitKey(1)
    if key & 0xFF == 27:  # Press Esc to exit
        # Stop the music when your program is done
        pygame.mixer.music.stop()
        break

cap.release()
cv2.destroyAllWindows()


dentro
[[[145  74   0]
  [145  74   0]
  [145  75   0]
  ...
  [166  86  15]
  [166  85  18]
  [164  85  18]]

 [[144  75   2]
  [144  75   2]
  [144  76   1]
  ...
  [170  88  11]
  [168  86  14]
  [168  86  14]]

 [[143  74   4]
  [143  74   4]
  [143  74   4]
  ...
  [178  91   3]
  [176  90   6]
  [174  91   6]]

 ...

 [[ 68  33  19]
  [ 68  33  19]
  [ 68  33  19]
  ...
  [220 156 131]
  [214 152 128]
  [211 149 125]]

 [[ 68  33  19]
  [ 68  33  19]
  [ 68  33  19]
  ...
  [215 151 126]
  [210 148 124]
  [206 144 120]]

 [[ 68  33  19]
  [ 68  33  19]
  [ 68  33  19]
  ...
  [213 149 124]
  [207 145 121]
  [203 141 117]]]
dentro
[[[145  74   0]
  [145  74   0]
  [145  75   0]
  ...
  [166  86  15]
  [166  85  18]
  [164  85  18]]

 [[144  75   2]
  [144  75   2]
  [144  76   1]
  ...
  [170  88  11]
  [168  86  14]
  [168  86  14]]

 [[143  74   4]
  [143  74   4]
  [143  74   4]
  ...
  [178  91   3]
  [176  90   6]
  [174  91   6]]

 ...

 [[ 68  33  19]
  [ 68  33  19]
  [ 68

In [108]:
import cv2

# Load the image
image = cv2.imread("blade.png")

if image is not None:
    # Display the image in a window with a specified name
    cv2.imshow("Blade Runner Image", image)

    # Wait for a key press and then close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Failed to load the image.")
